<img src="../pic/icon_exercise_header_p1.png" width="100%"> <br>

# ICON Community Interface ComIn - Practical Exercise Notebooks <br>  <br> Running ICON

In this notebook, we will set up a limited area mode (LAM) simulation and submit an ICON job on Levante.  
This notebook is used in all of the exercises (P1, P2 and P3). Make sure to rerun it completely from the beginning when you start with a new step to ensure there are no unwanted leftovers from the previous step.

## General Preparations for running a LAM simulation

 Base directory for ICON sources and binary:

In [ ]:
export SCRATCHDIR=/scratch/${USER::1}/$USER
export ICONDIR=${SCRATCHDIR}/icon

Absolute path to directory with plenty of space:

In [ ]:
export EXPDIR=${SCRATCHDIR}/icon_exercise_comin
if [ ! -d $EXPDIR ]; then
    mkdir -p $EXPDIR
fi

We start by preparing all the necessary data in the output directory. Note that the following block copies already prepared namelists and a sbatch script into the output directory.

In [ ]:
export SCRIPTDIR=$HOME/comin-training-exercises/exercise
cp $SCRIPTDIR/prepared/icon_master.namelist $EXPDIR/
cp $SCRIPTDIR/prepared/NAMELIST_ICON $EXPDIR/
cp $SCRIPTDIR/prepared/icon-lam.sbatch $EXPDIR/

In [ ]:
# directory with input grids and external data:
export GRIDDIR=/pool/data/ICON/ICON_training/exercise_lam/grids
# directory with initial data:
export DATADIR=/pool/data/ICON/ICON_training/exercise_lam/data_lam

cd ${EXPDIR}

# Link data needed for radiation
ln -sf ${ICONDIR}/externals/ecrad/data ecrad_data

# grid files: link to output directory
ln -sf ${GRIDDIR}/*.nc .
# data files
ln -sf ${DATADIR}/* .

# dictionaries for the mapping: DWD GRIB2 names <-> ICON internal names
ln -sf ${ICONDIR}/run/ana_varnames_map_file.txt .
ln -sf ${GRIDDIR}/../exercise_lam/map_file.latbc .

# For output: Dictionary for the mapping: names specified in the output nml <-> ICON internal names
ln -sf ${ICONDIR}/run/dict.output.dwd dict.output.dwd

## Adding a new output namelist

<div class="alert alert-success">
    <b style="color:#2d4b9b;">Exercise:</b> 
    Add the variable you have implemented into your plugin to the following output namelist on regular lon/lat grid.
</div>

In [ ]:
cat >> $EXPDIR/NAMELIST_ICON << EOF
! output_nml: specifies an output stream --------------------------------------
&output_nml
 filetype                    = 4                     ! netcdf
 dom                         = 1
 output_bounds               = 0., 10000000., 3600.  ! start, end, increment
 steps_per_file              = 1
 mode                        = 1
 include_last                = .FALSE.
 steps_per_file_inclfirst    = .FALSE.
 output_filename             = 'NWP_LAM'
 filename_format             = '<output_filename>_DOM<physdom>_<datetime2>'
 output_grid                 = .FALSE.
 remap                       = 1                     ! 1: remap to lat-lon grid
 reg_lon_def                 = 0.8,0.1,17.2
 reg_lat_def                 = 43.9,0.1,57.7
 ml_varlist                  = ????????
/
EOF

### Solution 

<code>
 ml_varlist                  = 'comin_process_id'
</code>

In [ ]:
cat >> $EXPDIR/NAMELIST_ICON << EOF
! output_nml: specifies an output stream --------------------------------------
&output_nml
 filetype                    = 4                     ! netcdf
 dom                         = 1
 output_bounds               = 0., 10000000., 3600.  ! start, end, increment
 steps_per_file              = 1
 mode                        = 1
 include_last                = .FALSE.
 steps_per_file_inclfirst    = .FALSE.
 output_filename             = 'NWP_LAM'
 filename_format             = '<output_filename>_DOM<physdom>_<datetime2>'
 output_grid                 = .FALSE.
 remap                       = 1                     ! 1: remap to lat-lon grid
 reg_lon_def                 = 0.8,0.1,17.2
 reg_lat_def                 = 43.9,0.1,57.7
 ml_varlist                  = "comin_process_id"
/
EOF

## Adding comin_nml 

In [ ]:
export SCRIPTDIR=$HOME/comin-training-exercises/exercise/scripts
cat >> $EXPDIR/NAMELIST_ICON << EOF
&comin_nml
   plugin_list(1)%name           = "comin_plugin"
   plugin_list(1)%plugin_library = "$ICONDIR/build/externals/comin/build/plugins/python_adapter/libpython_adapter.so"
   plugin_list(1)%options        = "$SCRIPTDIR/comin_plugin.py"
/
EOF

## The ICON batch job

---

Submit the job to the HPC cluster, using the Slurm command `sbatch`.

In [ ]:
export ICONDIR=$ICONDIR
cd $EXPDIR && sbatch --account $SLURM_JOB_ACCOUNT icon-lam.sbatch

---

*Author info: Deutscher Wetterdienst (DWD) 2024 ::  comin@icon-model.org. For a full list of contributors, see CONTRIBUTING in the root directory. License info: see LICENSE file.*